In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.9
#config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
df = pd.read_csv("correct_data1.csv",header=None)

In [4]:
df.shape

(591850, 2)

In [5]:
text = "\n".join(df[1])
print('corpus length:', len(text))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))# cutting the SMILES corpus into the sequences used for training
maxlen = 80
step = 1
text_train = "\n".join(df[1][200000:400000])
text_test = "\n".join(df[1][400000:420000])

corpus length: 29288558
total chars: 27


In [6]:
sentences = []
next_chars = []
for i in range(0, len(text_train) - maxlen, step):
    sentences.append(text_train[i: i + maxlen])
    next_chars.append(text_train[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Vectorization done')

nb sequences: 10386018
Vectorization...
Vectorization done


In [7]:
sentences = []
next_chars = []
for i in range(0, len(text_test) - maxlen, step):
    sentences.append(text_test[i: i + maxlen])
    next_chars.append(text_test[i + maxlen])
print('nb sequences:', len(sentences))
print('Vectorization...')
X_test = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_test = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X_test[i, t, char_indices[char]] = 1
    y_test[i, char_indices[next_chars[i]]] = 1
print('Vectorization done')

nb sequences: 1298978
Vectorization...
Vectorization done


In [8]:
X.shape

(10386018, 80, 27)

In [9]:
y.shape

(10386018, 27)

In [10]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, BatchNormalization, CuDNNLSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [11]:
model = Sequential([CuDNNLSTM(256, input_shape=(X.shape[1],X.shape[2]),return_sequences=True),
                   BatchNormalization(axis=1),
                   CuDNNLSTM(256,return_sequences=False),
                   BatchNormalization(axis=1),
                   Dense(y.shape[1]),
                   Activation('softmax')])
# model.add()
# model.add()
# model.add(_
# model.add()
# model.add()
# model.add()

filename = "checkpoints/cudnn-overfitting-bigdata-weights-improvement-02-0.5536.hdf5"
model.load_weights(filename)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
                  metrics=['accuracy'])


In [12]:
# define the checkpoint
filepath="checkpoints/cudnn-overfitting-bigdata-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
early_stop = keras.callbacks.EarlyStopping(monitor='loss', verbose=1, patience=7)
callbacks_list = [checkpoint]

In [13]:
l_history = model.fit(X, y, epochs=50, batch_size=1024, callbacks=callbacks_list)

Epoch 1/50
10386018/10386018 [==============================] - 2772s 267us/step - loss: 0.5342 - acc: 0.8116

Epoch 00001: loss improved from inf to 0.53422, saving model to checkpoints/cudnn-overfitting-bigdata-weights-improvement-01-0.5342.hdf5
Epoch 2/50
10386018/10386018 [==============================] - 2839s 273us/step - loss: 0.5203 - acc: 0.8164

Epoch 00002: loss improved from 0.53422 to 0.52033, saving model to checkpoints/cudnn-overfitting-bigdata-weights-improvement-02-0.5203.hdf5
Epoch 3/50
10386018/10386018 [==============================] - 2721s 262us/step - loss: 0.5096 - acc: 0.8202

Epoch 00003: loss improved from 0.52033 to 0.50959, saving model to checkpoints/cudnn-overfitting-bigdata-weights-improvement-03-0.5096.hdf5
Epoch 4/50
10386018/10386018 [==============================] - 2850s 274us/step - loss: 0.5011 - acc: 0.8232

Epoch 00004: loss improved from 0.50959 to 0.50107, saving model to checkpoints/cudnn-overfitting-bigdata-weights-improvement-04-0.5011.h

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(l_history.history).to_csv("history.csv")

In [ ]:
sed = np.zeros(maxlen, len(chars)), dtype=np.bool)


In [19]:
# pick a random seed
start = np.random.randint(0, len(X_test)-1)
pattern = X_test[start]
print ("Seed:")
print ("\"", ''.join([indices_char[value.argmax()] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern,(1,pattern.shape[0],pattern.shape[1]))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = indices_char[index]
    seq_in = [indices_char[value.argmax()]for value in pattern]
    print(result,end="")
    row = np.zeros(len(chars))
    row[index] = 1
    pattern = np.vstack([pattern,row])
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" 3(F)C(O)CC12C)OC(=O)C
Cc1oc(C=C2SC(=O)N(Cc3ccc4ccccc4c3)C2=O)cc1
CC1Cc2c(OCc3ccc "
cc3)c(C)nc4c(C=CC(=O)NCCO)sc2c5CCCC15
CC(C)CC(NC(=O)C(Cc1ccccc1)NC(=O)C(Cc2ccccc2)NC(=O)C(CCC(=O)N)NC(=O)C(CC(C)C)NC(=O)C(CCC(=O)N)NC(=O)C3CCCN3C(=O)C(CCCCN)NC(=O)C4CCCN4C(=O)C(N)CCCN=C(N)N)C(=O)N
CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(C)NC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(C)NC(=O)C(C)NC(=O)C(CC(C)C)NC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC(

In [14]:
model.evaluate(X_test,y_test, batch_size=1024)

1298978/1298978 [==============================] - 66s 51us/step


[0.5135894857326696, 0.8297646303400575]

In [ ]:
pattern.shape

In [ ]:
chars

In [ ]:
def plt_accuracy_curves(history, title):
  
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title(title)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
def plt_loss_curves(history, title):
  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
plt_accuracy_curves(history)

In [ ]:
plt_loss_curves(history)

In [ ]:
len("C(CCCNC(=N)N)NC(=O)")

In [ ]:
len("CCCCCCCCCCCCCCCCNc1ccc(cc1)C(=O)OCC(O)CO")